In [34]:
import polars as pl
from pathlib import Path
from providers.config import settings
import webbrowser
import random

pl.Config.set_fmt_str_lengths(200)

polars.config.Config

In [16]:
models = [
    "gemini-2.0-flash-exp",
    "gemini-1.5-pro",
    "gemini-1.5-flash",
    "gpt-4o-mini",
    "gpt-4o",
    "claude-3-5-sonnet-latest",
]

dfs = []
for model in models:
    df = pl.read_csv(f"./scores/{model}_scores.csv")
    dfs.append(df.rename({"score": model}))

merged_df = dfs[0]
for df in dfs[1:]:
    merged_df = merged_df.join(df, on="filename", how="full", coalesce=True)

merged_df = merged_df.with_columns(pl.col(pl.Float64).round(2))
merged_df

filename,gemini-2.0-flash-exp,gemini-1.5-pro,gemini-1.5-flash,gpt-4o-mini,gpt-4o,claude-3-5-sonnet-latest
str,f64,f64,f64,f64,f64,f64
"""31365_png.rf.c15a4717175cda592…",null,1.0,0.72,0.79,0.83,1.0
"""3066_png.rf.a271ccaade01ec4a7e…",1.0,0.9,1.0,1.0,1.0,1.0
"""42405_png.rf.29f7f349b16be7e94…",null,0.88,0.87,0.75,null,0.71
"""6464_png.rf.87df45257826205d38…",null,1.0,0.66,0.82,null,null
"""14587_png.rf.8ac51dfae9c3323be…",null,null,0.52,0.22,null,null
…,…,…,…,…,…,…
"""4369_png.rf.22210329dce81c468f…",null,0.94,null,0.33,0.32,0.62
"""20008_png.rf.11bd3ea6ad0610c46…",null,0.85,null,0.26,0.02,0.01
"""3794_png.rf.f3b9a9fce3f6f5b4e4…",null,0.58,null,0.08,null,0.5


In [74]:
def open_output_file(filename: str, model: str):
    url = settings.output_dir / f"{model}-raw" / filename
    assert url.exists(), f"File {url} does not exist"
    print(url)

    webbrowser.open(str(url))


def open_ground_truth_file(filename: str):
    url = settings.input_dir / "groundtruth" / filename
    assert url.exists(), f"File {url} does not exist"
    print(url)
    webbrowser.open(str(url))


def open_source_file(filename: str):
    url = settings.input_dir / "pdfs" / filename.replace(".html", ".pdf")
    assert url.exists(), f"File {url} does not exist"
    print(url)
    webbrowser.open(str(url))


random_filename = random.choice(merged_df.to_dicts())
print(random_filename["filename"])
print(random_filename["claude-3-5-sonnet-latest"])
open_output_file(random_filename["filename"], "gpt-4o-mini")
open_ground_truth_file(random_filename["filename"])
open_source_file(random_filename["filename"])


36278_png.rf.745f6e6efb75dd8a2247cad732e53033.html
None
/Users/sergey/Downloads/rd-tablebench/outputs/gpt-4o-mini-raw/36278_png.rf.745f6e6efb75dd8a2247cad732e53033.html
/Users/sergey/Downloads/rd-tablebench/groundtruth/36278_png.rf.745f6e6efb75dd8a2247cad732e53033.html
/Users/sergey/Downloads/rd-tablebench/pdfs/36278_png.rf.745f6e6efb75dd8a2247cad732e53033.pdf
